# 33. オーバーラップセグメントによる3段階カスケード枝刈り

## 目的

実験32で有効性が確認されたOverlap(8/4)をベースに、実験30と同様の3段階枝刈り方式を検証する。

## 3段階枝刈りフロー

```
全データ (400,000件)
    ↓ Step 1: Overlapセグメント一致フィルタ
候補 (~5,000件)
    ↓ Step 2: ハミング距離ソート
候補 (~500件)
    ↓ Step 3: コサイン類似度
Top-K 結果
```

## 検証するOverlap設定

| 設定 | セグメント幅 | ストライド | セグメント数 | バケットサイズ |
|------|------------|-----------|-------------|---------------|
| (8, 4) | 8bit | 4bit | 31 | 約1,562件 |
| (8, 2) | 8bit | 2bit | 61 | 約1,562件 |
| (16, 8) | 16bit | 8bit | 15 | 約8.7件 |
| (16, 4) | 16bit | 4bit | 29 | 約8.7件 |

## 評価指標

- Recall@5, @10, @20
- 各Stepでの候補数
- 処理時間

In [2]:
import sys
sys.path.insert(0, '..')

import numpy as np
from numpy.linalg import norm
import pandas as pd
import duckdb
from tqdm import tqdm
import time
from collections import defaultdict

from src.itq_lsh import ITQLSH, hamming_distance_batch

In [3]:
# パス設定
DB_PATH = '../data/experiment_400k.duckdb'
ITQ_MODEL_PATH = '../data/itq_model.pkl'

---

## 1. データ読み込み

In [4]:
# データ読み込み
print('データ読み込み中...')
conn = duckdb.connect(DB_PATH, read_only=True)

result = conn.execute("""
    SELECT id, embedding
    FROM documents
    ORDER BY id
""").fetchall()
conn.close()

doc_ids = np.array([r[0] for r in result])
embeddings = np.array([r[1] for r in result], dtype=np.float32)

print(f'読み込み完了: {len(doc_ids):,}件')
print(f'埋め込み shape: {embeddings.shape}')

データ読み込み中...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

読み込み完了: 400,000件
埋め込み shape: (400000, 1024)


In [5]:
# ITQモデル読み込みとハッシュ計算
print('ITQモデル読み込み中...')
itq = ITQLSH.load(ITQ_MODEL_PATH)

print('ITQハッシュ計算中...')
hashes = itq.transform(embeddings)
print(f'ハッシュ shape: {hashes.shape}')

ITQモデル読み込み中...
ITQハッシュ計算中...
ハッシュ shape: (400000, 128)


---

## 2. 共通ユーティリティ関数

In [6]:
def compute_ground_truth(query_embedding, all_embeddings, top_k=20):
    """ブルートフォースでGround Truthを計算"""
    query_norm = norm(query_embedding)
    all_norms = norm(all_embeddings, axis=1)
    cosines = (all_embeddings @ query_embedding) / (all_norms * query_norm + 1e-10)
    top_indices = np.argsort(cosines)[-top_k:][::-1]
    return top_indices


def evaluate_method(test_query_indices, predicted_results, ground_truths, top_k_values=[5, 10, 20]):
    """Recall評価"""
    recalls = {k: [] for k in top_k_values}
    
    for qi in test_query_indices:
        gt = ground_truths[qi]
        pred = predicted_results[qi]
        
        for k in top_k_values:
            gt_set = set(gt[:k])
            pred_set = set(pred[:k]) if len(pred) >= k else set(pred)
            recalls[k].append(len(gt_set & pred_set) / k)
    
    return {f'recall@{k}': np.mean(v) for k, v in recalls.items()}

In [7]:
# テスト用クエリを準備
rng = np.random.default_rng(42)
n_test_queries = 100
test_query_indices = rng.choice(len(embeddings), n_test_queries, replace=False)

print(f'テストクエリ数: {n_test_queries}')

# Ground Truth計算
print('Ground Truth計算中...')
ground_truths = {}
for qi in tqdm(test_query_indices, desc='GT計算'):
    ground_truths[qi] = compute_ground_truth(embeddings[qi], embeddings, top_k=20)
print('完了')

テストクエリ数: 100
Ground Truth計算中...


GT計算: 100%|██████████| 100/100 [00:33<00:00,  2.96it/s]

完了


---

## 3. オーバーラップセグメントインデックスの構築

In [8]:
def hash_to_overlap_segments(hash_array, segment_width=8, stride=4):
    """
    オーバーラップセグメントを生成
    
    Args:
        hash_array: (n_docs, 128) のハッシュ配列
        segment_width: セグメント幅（ビット数）
        stride: スライド幅（ビット数）
    
    Returns:
        segments: (n_docs, n_segments) の整数配列
        n_segments: セグメント数
    """
    n_docs, n_bits = hash_array.shape
    n_segments = (n_bits - segment_width) // stride + 1
    
    segments = []
    for i in range(n_segments):
        start = i * stride
        end = start + segment_width
        segment_bits = hash_array[:, start:end]
        
        # ビットを整数に変換
        powers = 2 ** np.arange(segment_width - 1, -1, -1)
        segment_int = np.sum(segment_bits * powers, axis=1)
        segments.append(segment_int)
    
    return np.column_stack(segments), n_segments


def build_overlap_index(segments):
    """
    オーバーラップセグメントのインデックスを構築
    """
    n_docs, n_segments = segments.shape
    index = {i: defaultdict(list) for i in range(n_segments)}
    
    for doc_idx in range(n_docs):
        for seg_idx in range(n_segments):
            seg_value = int(segments[doc_idx, seg_idx])
            index[seg_idx][seg_value].append(doc_idx)
    
    return index

In [9]:
# 複数のOverlap設定でインデックス構築
print('オーバーラップセグメントインデックス構築中...')

overlap_configs = [
    (8, 4),   # 8bit幅、4bitストライド → 31セグメント
    (8, 2),   # 8bit幅、2bitストライド → 61セグメント
    (16, 8),  # 16bit幅、8bitストライド → 15セグメント
    (16, 4),  # 16bit幅、4bitストライド → 29セグメント
]

overlap_data = {}

for width, stride in overlap_configs:
    key = (width, stride)
    segments, n_seg = hash_to_overlap_segments(hashes, width, stride)
    index = build_overlap_index(segments)
    
    # 統計
    bucket_sizes = [len(docs) for seg_idx in index for docs in index[seg_idx].values()]
    
    overlap_data[key] = {
        'segments': segments,
        'index': index,
        'n_segments': n_seg,
        'avg_bucket_size': np.mean(bucket_sizes),
    }
    
    print(f'  ({width}, {stride}): {n_seg}セグメント, 平均バケット: {np.mean(bucket_sizes):.1f}件')

オーバーラップセグメントインデックス構築中...
  (8, 4): 31セグメント, 平均バケット: 1562.5件
  (8, 2): 61セグメント, 平均バケット: 1562.5件
  (16, 8): 15セグメント, 平均バケット: 8.7件
  (16, 4): 29セグメント, 平均バケット: 8.7件


---

## 4. 3段階カスケード枝刈り関数の実装

In [10]:
def step1_overlap_filter(query_segments, segment_index, n_segments):
    """
    Step 1: Overlapセグメント一致によるフィルタリング（OR条件）
    
    いずれかのセグメントが一致したドキュメントを候補とする
    """
    candidates = set()
    
    for seg_idx in range(n_segments):
        seg_value = int(query_segments[seg_idx])
        if seg_value in segment_index[seg_idx]:
            candidates.update(segment_index[seg_idx][seg_value])
    
    return np.array(list(candidates))


def step2_hamming_filter(query_hash, candidate_indices, all_hashes, top_n):
    """
    Step 2: ハミング距離によるソートと絞り込み
    """
    if len(candidate_indices) == 0:
        return np.array([], dtype=np.int64)
    
    candidate_hashes = all_hashes[candidate_indices]
    distances = hamming_distance_batch(query_hash, candidate_hashes)
    
    top_n = min(top_n, len(candidate_indices))
    top_idx_in_candidates = np.argsort(distances)[:top_n]
    
    return candidate_indices[top_idx_in_candidates]


def step3_cosine_rank(query_embedding, candidate_indices, all_embeddings, top_k):
    """
    Step 3: コサイン類似度によるランキング
    """
    if len(candidate_indices) == 0:
        return np.array([], dtype=np.int64)
    
    candidate_embeddings = all_embeddings[candidate_indices]
    query_norm = norm(query_embedding)
    candidate_norms = norm(candidate_embeddings, axis=1)
    
    cosine_scores = (candidate_embeddings @ query_embedding) / (candidate_norms * query_norm + 1e-10)
    
    top_k = min(top_k, len(candidate_indices))
    top_idx_in_candidates = np.argsort(cosine_scores)[-top_k:][::-1]
    
    return candidate_indices[top_idx_in_candidates]

In [11]:
def cascade_search(
    query_embedding,
    query_hash,
    query_segments,
    segment_index,
    n_segments,
    all_hashes,
    all_embeddings,
    step1_limit=5000,
    step2_limit=500,
    top_k=20
):
    """
    3段階カスケード検索
    
    Step 1: Overlapセグメント一致 → step1_limit件
    Step 2: ハミング距離ソート → step2_limit件
    Step 3: コサイン類似度 → top_k件
    """
    timing = {}
    
    # Step 1: Overlapセグメント一致フィルタ
    t0 = time.time()
    step1_candidates = step1_overlap_filter(query_segments, segment_index, n_segments)
    step1_raw_count = len(step1_candidates)
    
    # Step1でstep1_limitを超えた場合はハミング距離で絞る
    if len(step1_candidates) > step1_limit:
        step1_candidates = step2_hamming_filter(
            query_hash, step1_candidates, all_hashes, step1_limit
        )
    timing['step1_ms'] = (time.time() - t0) * 1000
    step1_count = len(step1_candidates)
    
    # Step 2: ハミング距離ソート
    t0 = time.time()
    step2_candidates = step2_hamming_filter(
        query_hash, step1_candidates, all_hashes, step2_limit
    )
    timing['step2_ms'] = (time.time() - t0) * 1000
    step2_count = len(step2_candidates)
    
    # Step 3: コサイン類似度ランキング
    t0 = time.time()
    top_k_indices = step3_cosine_rank(
        query_embedding, step2_candidates, all_embeddings, top_k
    )
    timing['step3_ms'] = (time.time() - t0) * 1000
    
    timing['total_ms'] = timing['step1_ms'] + timing['step2_ms'] + timing['step3_ms']
    
    return {
        'top_k_indices': top_k_indices,
        'step1_raw_count': step1_raw_count,
        'step1_count': step1_count,
        'step2_count': step2_count,
        'timing': timing
    }

---

## 5. 3段階カスケード検索の評価

In [12]:
# 評価パラメータ設定
evaluation_configs = [
    # (overlap_config, step1_limit, step2_limit)
    # Overlap(8,4) ベース
    ((8, 4), 5000, 500),
    ((8, 4), 5000, 1000),
    ((8, 4), 10000, 500),
    ((8, 4), 10000, 1000),
    ((8, 4), 10000, 2000),
    
    # Overlap(8,2) - より多くのセグメント
    ((8, 2), 5000, 500),
    ((8, 2), 5000, 1000),
    ((8, 2), 10000, 1000),
    
    # Overlap(16,8) - 粗いセグメント
    ((16, 8), 2000, 500),
    ((16, 8), 5000, 500),
    
    # Overlap(16,4) - 中間
    ((16, 4), 3000, 500),
    ((16, 4), 5000, 500),
]

print(f'評価設定数: {len(evaluation_configs)}')

評価設定数: 12


In [13]:
# カスケード検索評価
print('=' * 80)
print('3段階カスケード検索評価')
print('=' * 80)

cascade_results = []

for overlap_config, s1_limit, s2_limit in tqdm(evaluation_configs, desc='設定'):
    width, stride = overlap_config
    data = overlap_data[overlap_config]
    segments = data['segments']
    index = data['index']
    n_seg = data['n_segments']
    
    predicted = {}
    step1_raw_counts = []
    step1_counts = []
    step2_counts = []
    times = []
    
    for qi in test_query_indices:
        result = cascade_search(
            embeddings[qi], hashes[qi], segments[qi],
            index, n_seg, hashes, embeddings,
            step1_limit=s1_limit, step2_limit=s2_limit, top_k=20
        )
        
        predicted[qi] = result['top_k_indices']
        step1_raw_counts.append(result['step1_raw_count'])
        step1_counts.append(result['step1_count'])
        step2_counts.append(result['step2_count'])
        times.append(result['timing']['total_ms'])
    
    recalls = evaluate_method(test_query_indices, predicted, ground_truths)
    
    cascade_results.append({
        'overlap': f'({width},{stride})',
        'width': width,
        'stride': stride,
        'n_segments': n_seg,
        'step1_limit': s1_limit,
        'step2_limit': s2_limit,
        'avg_step1_raw': np.mean(step1_raw_counts),
        'avg_step1': np.mean(step1_counts),
        'avg_step2': np.mean(step2_counts),
        **recalls,
        'avg_time_ms': np.mean(times)
    })

df_cascade = pd.DataFrame(cascade_results)
print('\n評価完了')

3段階カスケード検索評価


設定: 100%|██████████| 12/12 [00:23<00:00,  1.97s/it]


評価完了


In [14]:
# 結果表示
print('\n' + '=' * 120)
print('3段階カスケード検索結果サマリー')
print('=' * 120)

print(f'\n{"Overlap":>8} | {"S1 Lim":>7} | {"S2 Lim":>7} | {"S1 Raw":>8} | {"S1 Act":>7} | {"S2":>5} | {"R@5":>6} | {"R@10":>6} | {"R@20":>6} | {"Time":>8}')
print('-' * 120)

# Recall@10でソート
df_sorted = df_cascade.sort_values('recall@10', ascending=False)

for _, row in df_sorted.iterrows():
    print(f'{row["overlap"]:>8} | {row["step1_limit"]:>7} | {row["step2_limit"]:>7} | '
          f'{row["avg_step1_raw"]:>8.0f} | {row["avg_step1"]:>7.0f} | {row["avg_step2"]:>5.0f} | '
          f'{row["recall@5"]*100:>5.1f}% | {row["recall@10"]*100:>5.1f}% | {row["recall@20"]*100:>5.1f}% | '
          f'{row["avg_time_ms"]:>7.2f}ms')


3段階カスケード検索結果サマリー

 Overlap |  S1 Lim |  S2 Lim |   S1 Raw |  S1 Act |    S2 |    R@5 |   R@10 |   R@20 |     Time
------------------------------------------------------------------------------------------------------------------------
   (8,4) |   10000 |    2000 |    63764 |   10000 |  2000 |  90.4% |  90.0% |  86.6% |   23.46ms
   (8,2) |    5000 |    1000 |    86750 |    5000 |  1000 |  85.4% |  84.7% |  79.2% |   29.53ms
   (8,2) |   10000 |    1000 |    86750 |   10000 |  1000 |  85.4% |  84.7% |  79.2% |   30.57ms
   (8,4) |    5000 |    1000 |    63764 |    5000 |  1000 |  86.4% |  84.5% |  78.7% |   21.28ms
   (8,4) |   10000 |    1000 |    63764 |   10000 |  1000 |  86.4% |  84.5% |  78.7% |   22.28ms
   (8,2) |    5000 |     500 |    86750 |    5000 |   500 |  78.0% |  74.8% |  68.2% |   29.06ms
   (8,4) |   10000 |     500 |    63764 |   10000 |   500 |  77.8% |  74.4% |  67.9% |   21.53ms
   (8,4) |    5000 |     500 |    63764 |    5000 |   500 |  77.8% |  74.4% |  67.9% 

---

## 6. ベースライン（2段階検索）との比較

In [15]:
def two_stage_search(query_embedding, query_hash, all_hashes, all_embeddings, candidates, top_k):
    """従来の2段階検索（全件ハミング距離→コサイン）"""
    distances = hamming_distance_batch(query_hash, all_hashes)
    candidate_indices = np.argsort(distances)[:candidates]
    return step3_cosine_rank(query_embedding, candidate_indices, all_embeddings, top_k)

In [16]:
# ベースライン評価
print('ベースライン（2段階検索）評価中...')

baseline_results = []
for candidates in [500, 1000, 2000, 5000]:
    predicted = {}
    times = []
    
    for qi in tqdm(test_query_indices, desc=f'候補{candidates}'):
        t0 = time.time()
        predicted[qi] = two_stage_search(
            embeddings[qi], hashes[qi], hashes, embeddings,
            candidates=candidates, top_k=20
        )
        times.append((time.time() - t0) * 1000)
    
    recalls = evaluate_method(test_query_indices, predicted, ground_truths)
    baseline_results.append({
        'method': f'2段階({candidates})',
        'step2_limit': candidates,
        **recalls,
        'avg_time_ms': np.mean(times)
    })

df_baseline = pd.DataFrame(baseline_results)
print('\nベースライン結果:')
print(df_baseline.to_string(index=False))

ベースライン（2段階検索）評価中...


候補5000: 100%|██████████| 100/100 [00:04<00:00, 24.67it/s]


ベースライン結果:
   method  step2_limit  recall@5  recall@10  recall@20  avg_time_ms
 2段階(500)          500     0.788      0.750     0.6820    34.184358
2段階(1000)         1000     0.860      0.848     0.7950    35.529861
2段階(2000)         2000     0.916      0.913     0.8775    36.199548
2段階(5000)         5000     0.952      0.960     0.9530    40.268197


---

## 7. 最終比較

In [17]:
# 最終比較
print('\n' + '=' * 120)
print('最終比較: 3段階カスケード vs 2段階検索')
print('=' * 120)

print('\n■ 3段階カスケード検索（Recall@10上位5件）')
print(f'{"Overlap":>8} | {"S1":>7} | {"S2":>5} | {"R@10":>6} | {"削減率":>6} | {"Time":>8} | 備考')
print('-' * 90)

for _, row in df_sorted.head(5).iterrows():
    reduction = (400000 - row['avg_step1']) / 400000 * 100
    note = ''
    if row['recall@10'] >= 0.90:
        note = '★推奨'
    print(f'{row["overlap"]:>8} | {row["step1_limit"]:>7} | {row["step2_limit"]:>5} | '
          f'{row["recall@10"]*100:>5.1f}% | {reduction:>5.1f}% | '
          f'{row["avg_time_ms"]:>7.2f}ms | {note}')

print('\n■ 2段階検索（ベースライン）')
print(f'{"手法":>15} | {"R@10":>6} | {"削減率":>6} | {"Time":>8}')
print('-' * 60)

for _, row in df_baseline.iterrows():
    reduction = (400000 - row['step2_limit']) / 400000 * 100
    print(f'{row["method"]:>15} | {row["recall@10"]*100:>5.1f}% | {reduction:>5.1f}% | {row["avg_time_ms"]:>7.2f}ms')


最終比較: 3段階カスケード vs 2段階検索

■ 3段階カスケード検索（Recall@10上位5件）
 Overlap |      S1 |    S2 |   R@10 |    削減率 |     Time | 備考
------------------------------------------------------------------------------------------
   (8,4) |   10000 |  2000 |  90.0% |  97.5% |   23.46ms | 
   (8,2) |    5000 |  1000 |  84.7% |  98.8% |   29.53ms | 
   (8,2) |   10000 |  1000 |  84.7% |  97.5% |   30.57ms | 
   (8,4) |    5000 |  1000 |  84.5% |  98.8% |   21.28ms | 
   (8,4) |   10000 |  1000 |  84.5% |  97.5% |   22.28ms | 

■ 2段階検索（ベースライン）
             手法 |   R@10 |    削減率 |     Time
------------------------------------------------------------
       2段階(500) |  75.0% |  99.9% |   34.18ms
      2段階(1000) |  84.8% |  99.8% |   35.53ms
      2段階(2000) |  91.3% |  99.5% |   36.20ms
      2段階(5000) |  96.0% |  98.8% |   40.27ms


---

## 8. Step1での枝刈り効果分析

In [18]:
# Step1での枝刈り効果を分析
print('=' * 80)
print('Step1（Overlapセグメント一致）での枝刈り効果')
print('=' * 80)

print('\n■ Overlap設定別の候補数（Step1 Raw）')
for config, data in overlap_data.items():
    width, stride = config
    rows = df_cascade[df_cascade['overlap'] == f'({width},{stride})']
    if len(rows) > 0:
        avg_raw = rows['avg_step1_raw'].mean()
        reduction = (400000 - avg_raw) / 400000 * 100
        print(f'  Overlap({width},{stride}): 平均候補数 {avg_raw:,.0f}件, 削減率 {reduction:.1f}%')

print('\n■ 考察')
print('  - Overlap(8,4/8,2): 候補が多い（6-9万件）→ Step1でさらにハミング距離絞りが必要')
print('  - Overlap(16,8/16,4): 候補が少ない（2-3千件）→ Step1で大幅に絞れるが精度低下の可能性')

Step1（Overlapセグメント一致）での枝刈り効果

■ Overlap設定別の候補数（Step1 Raw）
  Overlap(8,4): 平均候補数 63,764件, 削減率 84.1%
  Overlap(8,2): 平均候補数 86,750件, 削減率 78.3%
  Overlap(16,8): 平均候補数 2,384件, 削減率 99.4%
  Overlap(16,4): 平均候補数 3,308件, 削減率 99.2%

■ 考察
  - Overlap(8,4/8,2): 候補が多い（6-9万件）→ Step1でさらにハミング距離絞りが必要
  - Overlap(16,8/16,4): 候補が少ない（2-3千件）→ Step1で大幅に絞れるが精度低下の可能性


---

## 9. 結論

In [19]:
# 最良設定の特定
best_cascade = df_sorted.iloc[0]
best_90plus = df_sorted[df_sorted['recall@10'] >= 0.90].iloc[0] if len(df_sorted[df_sorted['recall@10'] >= 0.90]) > 0 else None
baseline_2000 = df_baseline[df_baseline['step2_limit'] == 2000].iloc[0]

print('=' * 80)
print('結論')
print('=' * 80)

print(f'''
■ 実験目的
  Overlap(8/4)をベースに、実験30と同様の3段階枝刈り方式を検証

■ 最良設定（Recall@10）
  設定: Overlap{best_cascade["overlap"]}, Step1={best_cascade["step1_limit"]}件, Step2={best_cascade["step2_limit"]}件
  Recall@10: {best_cascade["recall@10"]*100:.1f}%
  Step1候補数: {best_cascade["avg_step1"]:.0f}件
  処理時間: {best_cascade["avg_time_ms"]:.2f}ms
''')

if best_90plus is not None:
    reduction = (400000 - best_90plus['avg_step1']) / 400000 * 100
    print(f'''■ Recall@10 ≥ 90%の最小コスト設定
  設定: Overlap{best_90plus["overlap"]}, Step1={best_90plus["step1_limit"]}件, Step2={best_90plus["step2_limit"]}件
  Recall@10: {best_90plus["recall@10"]*100:.1f}%
  Step1候補数: {best_90plus["avg_step1"]:.0f}件（削減率{reduction:.1f}%）
  処理時間: {best_90plus["avg_time_ms"]:.2f}ms
''')

print(f'''■ ベースライン（2段階検索）との比較
  2段階(2000): Recall@10 = {baseline_2000["recall@10"]*100:.1f}%, 処理時間 = {baseline_2000["avg_time_ms"]:.2f}ms
  3段階最良:   Recall@10 = {best_cascade["recall@10"]*100:.1f}%, 処理時間 = {best_cascade["avg_time_ms"]:.2f}ms
  
  ※ 2段階検索は全件（40万件）のハミング距離を計算
  ※ 3段階カスケードはStep1で候補を絞ってからハミング距離を計算
''')

結論

■ 実験目的
  Overlap(8/4)をベースに、実験30と同様の3段階枝刈り方式を検証

■ 最良設定（Recall@10）
  設定: Overlap(8,4), Step1=10000件, Step2=2000件
  Recall@10: 90.0%
  Step1候補数: 10000件
  処理時間: 23.46ms

■ ベースライン（2段階検索）との比較
  2段階(2000): Recall@10 = 91.3%, 処理時間 = 36.20ms
  3段階最良:   Recall@10 = 90.0%, 処理時間 = 23.46ms

  ※ 2段階検索は全件（40万件）のハミング距離を計算
  ※ 3段階カスケードはStep1で候補を絞ってからハミング距離を計算



---

## 10. 実験評価まとめ

### 実験目的
Overlap(8/4)をベースに、3段階カスケード枝刈り方式を検証し、DB負荷低減と精度のバランスを探る。

### 3段階枝刈りフロー

```
全データ (400,000件)
    ↓ Step 1: Overlapセグメント一致 → 6.4万件（84%削減）
候補 (step1_limit件)
    ↓ Step 2: ハミング距離ソート
候補 (step2_limit件)
    ↓ Step 3: コサイン類似度
Top-K 結果
```

### 検証した設定と結果

| Overlap | S1 Limit | S2 Limit | R@10 | Step1 Raw | 削減率 | 処理時間 | 評価 |
|---------|----------|----------|------|-----------|--------|----------|------|
| **(8,4)** | 10000 | **2000** | **90.0%** | 63,764 | 97.5% | 23.5ms | **★推奨** |
| (8,4) | 10000 | 1000 | 84.5% | 63,764 | 97.5% | 22.3ms | |
| (8,4) | 5000 | 1000 | 84.5% | 63,764 | 98.8% | 21.3ms | |
| (8,2) | 10000 | 1000 | 84.7% | 86,750 | 97.5% | 30.6ms | |
| (16,4) | 5000 | 500 | 46.8% | 3,308 | 99.2% | 2.3ms | 精度不足 |

### ベースライン（2段階検索）との比較

| 手法 | R@10 | 全件計算 | 処理時間 | 備考 |
|------|------|---------|----------|------|
| **3段階(8,4)/10000/2000** | **90.0%** | **不要** | **23.5ms** | **★推奨** |
| 2段階(2000) | 91.3% | 必要 | 36.2ms | ベースライン |
| 2段階(1000) | 84.8% | 必要 | 35.5ms | |
| 2段階(5000) | 96.0% | 必要 | 40.3ms | 高精度 |

### 重要な発見

1. **Step2のLimit（コサイン計算対象）が精度を決める**
   - S2=500件: R@10 ≈ 74-75%（不足）
   - S2=1000件: R@10 ≈ 84-85%（やや不足）
   - S2=2000件: R@10 ≈ 90%（十分）

2. **Overlap(8,4)の3段階カスケードが最適**
   - Step1でセグメント一致: 40万件 → 6.4万件（84%削減）
   - Step1でハミング距離絞り: 6.4万件 → 1万件
   - Step2でハミング距離絞り: 1万件 → 2000件
   - Step3でコサイン計算: 2000件 → Top-K

3. **Overlap(16,x)は枝刈りしすぎて精度低下**
   - 削減率99%以上だが、R@10は40%台
   - 候補数が少なすぎて良い結果を逃す

### シナリオ別推奨設定

| シナリオ | Overlap | S1 Limit | S2 Limit | R@10 | 処理時間 |
|---------|---------|----------|----------|------|----------|
| **精度重視** | (8,4) | 10000 | 2000 | 90.0% | 23.5ms |
| **バランス** | (8,4) | 5000 | 1000 | 84.5% | 21.3ms |
| **高速重視** | (8,4) | 5000 | 500 | 74.4% | 20.5ms |

### 結論

1. **3段階カスケード方式は有効**
   - 全件ハミング距離計算（40万件）を回避
   - 処理時間35%短縮（36ms → 23ms）

2. **Overlap(8,4) + S1=10000 + S2=2000が最適**
   - R@10 = 90.0%（2段階91.3%との差は1.3pt）
   - 処理時間23.5ms（2段階より35%高速）
   - Step1で97.5%削減してからハミング距離計算

3. **精度とコストのトレードオフ**
   - S2を増やせば精度向上（S2=2000で90%達成）
   - S2を減らせば高速化（S2=500で20ms）
   - 用途に応じて調整可能

### DBへの適用を考えた場合

| 処理 | 3段階カスケード | 2段階検索 |
|------|----------------|-----------|
| Step1: インデックス引き | O(1) × 31セグメント | - |
| Step1: ハミング距離計算 | 6.4万件 | **40万件** |
| Step2: ハミング距離計算 | 1万件 | - |
| Step3: ベクトル読み込み | **2000件** | **2000件** |
| Step3: コサイン計算 | 2000件 | 2000件 |

**結論: 3段階カスケードはハミング距離計算を6分の1に削減し、DBからのベクトル読み込みも同等に抑えられる。**

### 次のステップ

- DuckDBでのセグメントインデックス構築と検索の実装
- 実際の外部クエリでの評価
- インデックスサイズと構築時間の計測